In [58]:
import csv
from datetime import datetime, timezone
import requests
csv_list = ["redis.redis.csv", 'microsoft.vscode.csv'] # ["redis.redis.csv"]


##### feature index ##########
pr_author_login_idx = 0  ### delete
pr_author_name_idx = 1  ### replace with race
pr_create_at_idx = 2  ### replace with notice time
pr_merged_at_idx = 3  ### replace with review time
pr_changed_files_idx = 4
pr_LOC_idx = 5
pr_reviewers_idx = 6   #### delete for hypo 1-4 , change for hypo 5
pr_first_review_at_idx = 7  ### delete
pr_review_commits_idx = 8
pr_review_comments_idx = 9
pr_commits_idx = 10

repo_stars_idx = 11
repo_created_at_idx = 12  ####  replace with repo time
repo_external_contribution_idx = 13
repo_team_size_idx = 14

user_followers_idx = 15
user_created_at_idx = 16   ### replace with user time
user_experience_idx = 17
user_repo_experience_idx = 18
user_main_member_idx = 19
user_watched_idx = 20
user_succ_rate_idx = 21

def set_string(reveiewers):
    
    reveiewers = reveiewers[1:-1]
    reveiewer_list = reveiewers.split(",")
    res = []
    for name in reveiewer_list:
        res.append(name[1:-1])
    return res

######### get the dictionary for user-name : race
race_dic = {} # key name, value race

for file_name in csv_list:
    f_read = open("./csv_files/" + file_name, 'r')
    with f_read:
        reader = csv.reader(f_read)
        count = 0
        for read_row in reader:
            if count == 0:
                count += 1
                continue
            
            author_name = read_row[pr_author_name_idx]
            race_dic[author_name] = "Unkonwn"
            for reviewer_name in set_string(read_row[pr_reviewers_idx]):
                race_dic[reviewer_name] = "Unkonwn"
#print(race_dic.keys())

for name in race_dic:
    full_name = name.replace(" ", "%20")
    #print(full_name) White API
    ethnic_req = requests.get("http://www.name-prism.com/api_token/eth/json/4868133984c85190/"+full_name).json()
    for race, value in ethnic_req.items():
        if value >= 0.8 and (race == "API" or race == "White" or race == "Hispanic"):
            race_dic[name] = race
            break
    time.sleep(0.1)

print(len(race_dic))

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
##### build the data for hypo 1-4 ##########

### the new variables for model
pr_head1 = ['pr_author_race','pr_notice_time','pr_review_time','pr_changed_files',
    'pr_LOC','pr_review_commits','pr_review_comments','pr_commits'] # 8


repo_head = ['repo_stars','repo_create_time','repo_external_contribution','repo_team_size'] # 4

user_head = ['user_followers','user_dev_time','user_experience', 'user_repo_experience','user_main_member',
             'user_watched','user_succ_rate'] # 7

csv_head1 = pr_head1 + repo_head + user_head

length = len(csv_head)
cur_time = datetime.now(timezone.utc)


with open("combine_data.csv", 'w') as f_write:
    csv_write = csv.writer(f_write)
    csv_head = csv_head1
    csv_write.writerow(csv_head)
    
    for file_name in csv_list:
        f_read = open("./csv_files/" + file_name, 'r')
        with f_read:
            reader = csv.reader(f_read)
            count = 0
            for read_row in reader:
                if count == 0:
                    count += 1
                    continue
                write_row = []

                
                ######## calculate the time difference ##########
                created_at = datetime.strptime(read_row[pr_create_at_idx], "%Y-%m-%dT%H:%M:%S%z")
                merged_at = datetime.strptime(read_row[pr_merged_at_idx], "%Y-%m-%dT%H:%M:%S%z") 
                review_at =  datetime.strptime(read_row[pr_first_review_at_idx] + "Z", "%Y-%m-%d %H:%M:%S%z") 
                repo_created_at = datetime.strptime(read_row[repo_created_at_idx], "%Y-%m-%dT%H:%M:%S%z") 
                user_created_at = datetime.strptime(read_row[user_created_at_idx], "%Y-%m-%dT%H:%M:%S%z") 
                
                pr_notice_time = merged_at - review_at
                pr_review_time = review_at - created_at
                repo_create_time = cur_time - repo_created_at
                user_create_time = cur_time - user_created_at
                ######### get the author race ##########
                author_name = read_row[pr_author_name_idx]
                pr_author_race = race_dic[author_name]

                ## pr features
                write_row.append(pr_author_race)
                write_row.append(pr_notice_time.days)
                write_row.append(pr_review_time.days)
                write_row.append(read_row[pr_changed_files_idx])
                write_row.append(read_row[pr_LOC_idx])
                write_row.append(read_row[pr_review_commits_idx])
                write_row.append(read_row[pr_review_comments_idx])
                write_row.append(read_row[pr_commits_idx])
                
                ## repo features
                write_row.append(read_row[repo_stars_idx])
                write_row.append(repo_create_time.days)
                write_row.append(read_row[repo_external_contribution_idx])
                write_row.append(read_row[repo_team_size_idx])

                ## user features
                write_row.append(read_row[user_followers_idx])
                write_row.append(user_create_time.days)
                write_row.append(read_row[user_experience_idx])
                write_row.append(read_row[user_repo_experience_idx])
                write_row.append(read_row[user_main_member_idx])
                write_row.append(read_row[user_watched_idx])
                write_row.append(read_row[user_succ_rate_idx])
                
                csv_write.writerow(write_row)
                

In [55]:
##### build the data for hypo 5 ##########
pr_head5 = ['pr_author_race','pr_notice_time','pr_review_time','pr_changed_files',
    'pr_LOC','pr_reviewer_race','pr_review_commits','pr_review_comments','pr_commits'] # 9

csv_head5 = pr_head5 + repo_head + user_head

with open("combine_data_h5.csv", 'w') as f_write:
    csv_write = csv.writer(f_write)
    csv_head = csv_head5
    csv_write.writerow(csv_head)
    
    for file_name in csv_list:
        f_read = open("./csv_files/" + file_name, 'r')
        with f_read:
            reader = csv.reader(f_read)
            count = 0
            for read_row in reader:
                if count == 0:
                    count += 1
                    continue
                write_row = []

                
                ######## calculate the time difference ##########
                created_at = datetime.strptime(read_row[pr_create_at_idx], "%Y-%m-%dT%H:%M:%S%z")
                merged_at = datetime.strptime(read_row[pr_merged_at_idx], "%Y-%m-%dT%H:%M:%S%z") 
                review_at =  datetime.strptime(read_row[pr_first_review_at_idx] + "Z", "%Y-%m-%d %H:%M:%S%z") 
                repo_created_at = datetime.strptime(read_row[repo_created_at_idx], "%Y-%m-%dT%H:%M:%S%z") 
                user_created_at = datetime.strptime(read_row[user_created_at_idx], "%Y-%m-%dT%H:%M:%S%z") 
                
                pr_notice_time = merged_at - review_at
                pr_review_time = review_at - created_at
                repo_create_time = cur_time - repo_created_at
                user_create_time = cur_time - user_created_at
                ######### get the author and reviewer race ##########
                author_name = read_row[pr_author_name_idx]
                pr_author_race = race_dic[author_name]
                
                reviewer_list = set_string(read_row[pr_reviewers_idx])
                if len(reviewer_list) != 1:
                    continue
                reviewer_race = race_dic[reviewer_list[0]]
                
                ## pr features
                write_row.append(pr_author_race)
                write_row.append(pr_notice_time.days)
                write_row.append(pr_review_time.days)
                write_row.append(read_row[pr_changed_files_idx])
                write_row.append(read_row[pr_LOC_idx])
                write_row.append(reviewer_race)
                write_row.append(read_row[pr_review_commits_idx])
                write_row.append(read_row[pr_review_comments_idx])
                write_row.append(read_row[pr_commits_idx])
                
                ## repo features
                write_row.append(read_row[repo_stars_idx])
                write_row.append(repo_create_time.days)
                write_row.append(read_row[repo_external_contribution_idx])
                write_row.append(read_row[repo_team_size_idx])

                ## user features
                write_row.append(read_row[user_followers_idx])
                write_row.append(user_create_time.days)
                write_row.append(read_row[user_experience_idx])
                write_row.append(read_row[user_repo_experience_idx])
                write_row.append(read_row[user_main_member_idx])
                write_row.append(read_row[user_watched_idx])
                write_row.append(read_row[user_succ_rate_idx])
                
                csv_write.writerow(write_row)